<a href="https://colab.research.google.com/github/stmulugheta/20223-ML-DL-models/blob/master/webscraping_shabait.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WebScraping for "https://shabait.com"

Initial support: Maria Fisher: roubadilhar = I have created this work from Portugues roubar (still) + compartilhar (share) = RoubaDilhar 3:-) 

I have roubadilhado* this codes from this tutorial (author : Jawakar)
https://medium.com/@jawakarselvavinayagam/scrape-data-from-web-to-csv-in-minutes-dfbaa1e8775d

# importning Libraries

In [1]:
!pip install autoscraper
!pip install pandas



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from autoscraper import AutoScraper

In [3]:
shabait = "https://shabait.com/"
wanted_list = ['Recent Posts ' ]

In [4]:
wanted_list

['Recent Posts ']

In [5]:
scraper = AutoScraper()
result = scraper.build(shabait, wanted_list)
data = scraper.get_result_similar(shabait, grouped=True)
print(data)

{'rule_4e5j': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Recent Posts'], 'rule_il85': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Stay With Us', 'Recent Posts']}


 Python dictionary with dynamically assigned keys
{'rule_rxxa': ['Recent'], 'rule_5s6n': ['Categories']}


In [6]:
key = list(data.keys())
print(key)
#output
{'rule_3no9': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Recent Posts'], 'rule_d5xd': ['Recent Posts', 'Newspaper - Haddas Ertra', 'Newspapers - Eritrea Haddas', 'Newspapers - Eritrea Alhaditha', 'Newspapers - Eritrea Profile', 'Recent Posts']}
scraper.save('shabait')

['rule_4e5j', 'rule_il85']


In [7]:
# Give it a file path
scraper.save('shabait')

In [8]:
# specify the path
scraper.load("shabait")
result = scraper.get_result_similar("https://shabait.com/", group_by_alias=True)

In [9]:
result

{'': ['Recent Posts',
  'Recent Posts',
  'Newspaper - Haddas Ertra',
  'Newspaper - Haddas Ertra',
  'Newspapers - Eritrea Haddas',
  'Newspapers - Eritrea Haddas',
  'Newspapers - Eritrea Alhaditha',
  'Newspapers - Eritrea Alhaditha',
  'Newspapers - Eritrea Profile',
  'Newspapers - Eritrea Profile',
  'Stay With Us',
  'Recent Posts',
  'Recent Posts']}

In [10]:
#!pip install pandas

In [11]:
import pandas as pd

df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in result.items() ]))
# Converting our DataFrame to CSV file
df.to_csv('shabait.csv')

In [12]:
df.head(10)

,
0,Recent Posts
1,Recent Posts
2,Newspaper - Haddas Ertra
3,Newspaper - Haddas Ertra
4,Newspapers - Eritrea Haddas
5,Newspapers - Eritrea Haddas
6,Newspapers - Eritrea Alhaditha
7,Newspapers - Eritrea Alhaditha
8,Newspapers - Eritrea Profile
9,Newspapers - Eritrea Profile


In [13]:
df.tail(10)

,
3,Newspaper - Haddas Ertra
4,Newspapers - Eritrea Haddas
5,Newspapers - Eritrea Haddas
6,Newspapers - Eritrea Alhaditha
7,Newspapers - Eritrea Alhaditha
8,Newspapers - Eritrea Profile
9,Newspapers - Eritrea Profile
10,Stay With Us
11,Recent Posts
12,Recent Posts


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0           13 non-null     object
dtypes: object(1)
memory usage: 232.0+ bytes


In [15]:
df.describe()

,
count,13
unique,6
top,Recent Posts
freq,4


# To be continued according to the need